#  Dashboard

Navigate the cosmos of machine learning as we unveil the ultimate chapter of our journey. Equipped with insights harvested from prior explorations, we engineer a vessel—a machine learning pipeline—that sails through the currents of data to predict Falcon 9 first stage landing outcomes. With the precision of starship captains, we standardize, split, and scrutinize the data. Navigating the hyperparameter space, we align Support Vector Machines, Classification Trees, and Logistic Regression with our predictive quest. This voyage culminates in the unveiling of the methodology that charts the truest course to predictive success, empowering us to unveil the cost of triumphant launch in the realm of space exploration.

In [16]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("SpaceX_Success_Forecast.csv")
max_payload = spacex_df['PayloadMass'].max()
min_payload = spacex_df['PayloadMass'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                #  Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                     {'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': 'VAFB SLC 4E', 'value': 'VAFB SLC 4E'},
                                                     {'label': 'KSC LC 39A', 'value': 'KSC LC 39A'},
                                                     {'label': 'CCAFS SLC 40', 'value': 'CCAFS SLC 40'}
                                                     ],
                                             value='ALL',
                                             placeholder='Select a Launch Site here',
                                             searchable=True
                                             ),
                                html.Br(),

                                # Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload, max_payload]
                                                ),

                                # Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='Class', 
        names='LaunchSite', 
        title='Success Count for all launch sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['LaunchSite']== entered_site]
        filtered_df=filtered_df.groupby(['LaunchSite','Class']).size().reset_index(name='class count')
        fig=px.pie(filtered_df,values='class count',names='Class',title=f"Total Success Launches for site {entered_site}")
        return fig


# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])
def scatter(entered_site,payload):
    filtered_df = spacex_df[spacex_df['PayloadMass'].between(payload[0],payload[1])]
    # thought reusing filtered_df may cause issues, but tried it out of curiosity and it seems to be working fine
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='PayloadMass',y='Class',color='BoosterVersion',title='Success count on Payload mass for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['LaunchSite']==entered_site],x='PayloadMass',y='Class',color='BoosterVersion',title=f"Success count on Payload mass for site {entered_site}")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


